In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import time
import os
import copy

from dgl.geometry import farthest_point_sampler
import open3d as o3d

from utils import fps, depth2fgpcd, pcd2pix, fps_np, downsample_pcd

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
env = 'carrots'
folder_path = f'ptcl_data/{env}'

In [3]:
pcd_all_list = []
extrinsic_matrixs = []
global_scale = 24

for i in range(1, 5):
    # load data
    cam_view = 'view_{}'.format(i)
    dir_path = os.path.join(folder_path, cam_view)
    
    # obs = np.load(os.path.join(dir_path, 'obs.npy'))
    camera_intrinsic_params = np.load(os.path.join(dir_path, 'camera_intrinsic_params.npy')) # [fx, fy, cx, cy]
    camera_extrinsic_matrix = np.load(os.path.join(dir_path, 'camera_extrinsic_matrix.npy'))
    print('i:', i)
    print('camera_intrinsic_params\n', camera_intrinsic_params)
    print('camera_extrinsic_matrix\n', camera_extrinsic_matrix)
    extrinsic_matrixs.append(camera_extrinsic_matrix)
    
    pcd = o3d.io.read_point_cloud(os.path.join(dir_path, 'fgpcd.pcd'))
    pcd_array = np.asarray(pcd.points)
    print(pcd_array.shape, pcd_array.mean(axis=0))
    
    pcd_all_list.append(pcd)
    

i: 1
camera_intrinsic_params
 [869.11683083 869.11683083 360.         360.        ]
camera_extrinsic_matrix
 [[-4.3711388e-08  0.0000000e+00 -1.0000000e+00  5.5635513e-07]
 [-7.0710677e-01  7.0710677e-01  3.0908620e-08  0.0000000e+00]
 [ 7.0710677e-01  7.0710677e-01 -3.0908620e-08 -1.8000000e+01]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]
(62234, 3) [ 24.7975479  -25.138006    -0.17989309]
i: 2
camera_intrinsic_params
 [869.11683083 869.11683083 360.         360.        ]
camera_extrinsic_matrix
 [[  1.           0.           0.           0.        ]
 [  0.           0.70710677  -0.70710677   0.        ]
 [  0.           0.70710677   0.70710677 -18.        ]
 [  0.           0.           0.           1.        ]]
(63303, 3) [  0.13475381 -25.14086781 -24.79815252]
i: 3
camera_intrinsic_params
 [869.11683083 869.11683083 360.         360.        ]
camera_extrinsic_matrix
 [[ 1.1924881e-08  0.0000000e+00  1.0000000e+00  1.5177896e-07]
 [ 7.0710677e-01  7.0710677e-01 

In [4]:
o3d.visualization.draw_geometries(pcd_all_list)

In [5]:
pcd_all = o3d.geometry.PointCloud()
pcd_all = [pcd_all_list[0]]
for point_id in range(2):
    trans = extrinsic_matrixs[point_id] @ np.linalg.inv(extrinsic_matrixs[0])
    pcd_all_list[point_id].transform(trans)
    pcd_all.append(pcd_all_list[point_id])

In [6]:
o3d.visualization.draw_geometries(pcd_all)

In [7]:
o3d.visualization.draw_geometries([pcd_all_list[1]])

### Merge 2

In [8]:
source_0 = copy.deepcopy(pcd_all_list[0]) 
source_1 = copy.deepcopy(pcd_all_list[1])

In [9]:
extr_0 = extrinsic_matrixs[0].copy()
extr_1 = extrinsic_matrixs[1].copy()

In [10]:
o3d.visualization.draw_geometries([source_0, source_1])

In [11]:
source_0.transform(extr_0)

PointCloud with 62234 points.

In [12]:
o3d.visualization.draw_geometries([source_0, source_1])

In [13]:
source_0.transform(extr_0 @ np.linalg.inv(extr_1))

PointCloud with 62234 points.

In [ ]:
o3d.visualization.draw_geometries([source_0, source_1])